# Time series - multivariate prediction, data preprocessing

In this example, we will use `pandas` to load and prepare dataset for multivariate prediction.

The dataset is from Prague airport daily weather measurements.

In [ ]:
import numpy as np
import os
from datetime import date
import re

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.options.display.max_rows = 30

First, let's prepare some constants

In [ ]:
days = {str(y) for y in range(1, 32)}
months = dict(zip(
    ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'),
    range(1, 13)
))            
years = {str(y) for y in range(1996, 2018)}

rain_flags = ['Thunderstorm', 'Rain', 'Hail', 'Snow']

def triple(col_name):
    return [col_name + '[hi]', col_name + '[av]', col_name + '[lo]']

columns = ['Temp', 'DPoint', 'Humidity', 'SLPress', 'Vis', 'Wind']
all_columns = [t for col in columns for t in triple(col)] + ['Precip[sum]', 'Precip[0-1]']

def floatOrNaN(token):
    try:
        return float(token)
    except ValueError:
        return np.NaN

Here's the set of all columns for the whole database.

In [ ]:
print(all_columns)

Now, let's prepare a method for loading dataset into pandas dataframe:

In [ ]:
def load_to_dataframe(file_path):
    print('Loading: %s... ' % file_path, end='')
    with open(file_path) as file:
        lines = [line for line in file]

    if not lines[-1].endswith('\n'):
        lines[-1] = lines[-1] + '\n'

    tokens = re.split('[\t\n]', lines[0])[:-1]    
    if tokens[0] not in years:
        raise ValueError('Year expected: %s' % tokens[0])    
    current_year = tokens[0]

    tokens = re.split('[\t\n]', lines[1])[:-1]    
    if tokens[0] not in months:
        raise ValueError('Month expected: %s' % tokens[0])    
    current_month = tokens[0]

    dates = []
    values = []

    for line in lines[2:]:        
        tokens = re.split('[\t\n]', line)[:-1]    
        if tokens[0] in years:
            current_year = tokens[0]
        elif tokens[0] in months:
            current_month = tokens[0]
        elif tokens[0] in days:
            dates.append(date(int(current_year), months[current_month], int(tokens[0])))            
            has_rain = any(rf in tokens[-1] for rf in rain_flags)
            numbers = np.array([floatOrNaN(token) for token in tokens[1:-1]] + [1 if has_rain else 0])
            if numbers.shape[0] != 20:
                raise ValueError('Invalid row: %s %s' % (current_month, " ".join(tokens)))
            values.append(numbers)
        else:
            raise ValueError('Unexpected token: %s' % tokens[0])
            
    try:
        df = pd.DataFrame(np.stack(values), columns=all_columns, index=pd.DatetimeIndex(dates, freq='D', verify_integrity=True))
        print("%d rows." % len(values))
        return df
    except:
        print('%d rows. Warning! missing rows.' % len(values))
        return pd.DataFrame(np.stack(values), columns=all_columns, index=pd.DatetimeIndex(dates, freq='D', verify_integrity=False))

Load the dataset into a list of dataframes, one dataframe per year:

In [ ]:
PATH = '../data/weather/LKPR/'
files = sorted([filenames for dirpath, dirnames, filenames in os.walk(PATH)][0])

df_years = []

for file in sorted(files):
    df_years.append(load_to_dataframe(PATH + file))

We have some missing values (more then a half year gap) in the data. 

Let's take only data after the gap and make one long sequence out of all consecutive years

In [ ]:
df = pd.concat(df_years[5:])

The data needs to be normalized (again, column-wise normalization):

In [ ]:
df_min = df.min()
df_max = df.max()
df_norm = (df - df_min) / (df_max - df_min)

data_bounds = pd.concat((df_min, df_max), axis=1)
data_bounds

## Data selection

For our prediction we are going to select the following columns:

- **average temperature** (cont. value)
- **averate temperature of the dew point** (cont. value)
- **sea level pressure** (cont. value)
- **precipitation** (binary yes/no)

Notice that we would rather use rolling average of the values (except for precipitation) to mitigate hidden variables (sunlight etc.)

In [ ]:
selected_columns = ['Temp[av]', 'DPoint[av]', 'SLPress[av]']
selected_df = pd.concat((df_norm[c] for c in selected_columns), axis=1)
average = selected_df.rolling(window=5, center=True).mean()

Let's see our data for the last year.

In [ ]:
start_date = '2017-01-01'
end_date = '2017-12-31'

plt.figure(figsize = (25, 6))
plt.plot(selected_df[start_date:end_date], 'silver')
plt.plot(average[start_date:end_date])
plt.plot(df_norm['Precip[0-1]'][start_date:end_date], 'oc')
plt.show()

Finally, concatenate continuous value columns with precipitaion and store the dataset for later use.

In [ ]:
training_df = pd.concat((average, df_norm['Precip[0-1]']), axis=1).dropna()
training_df

In [ ]:
training_df.to_pickle('../data/weather/ts_temp_dp_press.p')
data_bounds.to_pickle('../data/weather/data_bounds.p')